In [1]:
import os

In [2]:
%pwd

'd:\\End-to-End-Wine-Quality-predidection\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\End-to-End-Wine-Quality-predidection'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir:Path
    test_data_path:Path
    model_path:Path
    all_param:dict
    metrics_file_name:Path
    target_column:str

In [6]:
#Configuration Manager

from ML_Project.constants import *
from ML_Project.entity.config_entity import DataIngestionConfig, DataTransformationConfig, DataValidationConfig, ModelTrainerConfig
from ML_Project.utils.common import create_directories


class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath= SCHEMA_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        create_directories([Path(self.config.artifact_root)])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
    
        config = self.config.data_ingestion
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir,
            artifact_dir=self.config.artifact_root
        )
       
        return data_ingestion_config
    
    def get_data_validation_config(self) -> DataValidationConfig:
        config=self.config.data_validation
        root_dir=Path(config.root_dir)
        create_directories([root_dir])
        data_validation_config=DataValidationConfig(
            root_dir=Path(config.root_dir),
            unzip_data_dir=Path(config.unzip_data_dir),
            STATUS_FILE=Path(config.STATUS_FILE),
            all_schema=self.schema
        )
        return data_validation_config
    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config=self.config.data_transformation
        root_dir=Path(config.root_dir)
        create_directories([root_dir])
        data_transformation_config=DataTransformationConfig(
            root_dir=root_dir,
            data_path=Path(config.data_path)
        )
        return data_transformation_config
    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config=self.config.model_trainer
        params=self.params.Elasticsearch
        schema=self.schema.TARGET_COLUMN
        model_trainer_config=ModelTrainerConfig(
            root_dir=Path(config.root_dir),
            train_data_path=Path(config.train_data_path),
            test_data_path=Path(config.test_data_path),
            model_name=config.model_name,
            alpha=params.alpha,
            l1_ratio=params.l1_ratio,
            target_column=schema.name
        )
        return model_trainer_config

    def get_model_evaluation_config(self)-> ModelEvaluationConfig:
        config=self.config.model_evaluation
        create_directories([Path(config.root_dir)])
        model_evaluation_config=ModelEvaluationConfig(
            root_dir=Path(config.root_dir),
            test_data_path=Path(config.test_data_path),
            model_path=Path(config.model_path),
            all_param=self.params,
            metrics_file_name=Path(config.metrics_file_name),
            target_column=self.schema.TARGET_COLUMN

        )
        return model_evaluation_config








In [17]:
import joblib
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score

from ML_Project.utils.common import save_json
import pandas as pd

class ModelEvaluation :
    def __init__(self,config:ModelEvaluationConfig):
        self.config=config


    def eval_metrics(self):
        
        X_test=pd.read_csv(self.config.test_data_path).drop(columns=[str(self.config.target_column.name)],axis=1)
        y_test=pd.read_csv(self.config.test_data_path)[[str(self.config.target_column.name)]]
        model_evaluation=joblib.load(self.config.model_path)
        predicted_results=model_evaluation.predict(X_test)


        mse=mean_squared_error(y_test,predicted_results)
        mae=mean_absolute_error(y_test,predicted_results)
        r2=r2_score(y_test,predicted_results)
        data={
            "mse":mse,
            "mae":mae,
            "r2":r2
        }
    #~Save Json
        save_json(data,self.config.metrics_file_name)
    

In [18]:

config=ConfigurationManager()
model_evaluation_config=config.get_model_evaluation_config()
model_evaluation=ModelEvaluation(config=model_evaluation_config)
model_evaluation.eval_metrics()


[2025-10-07 18:23:11,950] :INFO:common: Created directory at: artifacts :
[2025-10-07 18:23:11,951] :INFO:common: Created directory at: artifacts\model_evaluation :
[2025-10-07 18:23:12,017] :INFO:common: created json file at :artifacts\model_evaluation\metrics.json :
